# Showcase 2-Stage

### Load Model & Dataset

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.models.simple_3d_model import LinearModel
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.eval.accuracies import keypoint_depth_pck, get_max_preds

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    reduced_limb_ranges = [[0,4],[4,7],[7,10],[10,13],[13,16],[16,19],[19,22],[22,25],[25,28]]
    if len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line
    elif len(points) == 28:
        points = np.array(points).T
        for num in range(len(reduced_limb_ranges)):
            visible_limb = []
            for x in range(reduced_limb_ranges[num][0],reduced_limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [ ]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [ ]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,_ ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Load Models
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model_2d = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model_2d = PoseResNet(block_class, layers, 62)

model_2d = model_2d.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model_2d.load_state_dict(torch.load(path))
model_2d.eval()


In [ ]:
if reduceKeypoints:
    model_lift = LinearModel(2*28,28,1024) # Linear size 512 looked promising 256 too small
else:
    model_lift = LinearModel(2*62,62,512)

model_lift = model_lift.to(device)
path = "../models/lifting_depth/lifting_depth_model_20220323-150326_128"
model_lift.load_state_dict(torch.load(path))
model_lift.eval()

In [ ]:
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader:
        image = data['file_name']
        X = data['image'].to(device, dtype=torch.float)
        y = data['key_points_3D'].to(device, dtype=torch.float)
        vis = data['visibility']
        
        
        pred_heatmap =  model_2d(X)
        kps2d,_ = get_max_preds(pred_heatmap.detach().cpu().numpy())
        kps2d = (torch.from_numpy(kps2d)-torch.stack([torch.Tensor(sungaya_dataset.means_2d)]*len(kps2d)))/torch.stack([torch.Tensor(sungaya_dataset.std_2d)]*len(kps2d))

        kps2d = kps2d.reshape((len(kps2d), 56))
        pred = model_lift(kps2d.to(device))
        break

In [ ]:
import copy
sample =10

prediction = pred[sample].cpu().numpy()
actual = (y[sample].cpu()).numpy()
visibility_mask = vis[sample]

mean = sungaya_dataset.means_3d[:,2]
std = sungaya_dataset.std_3d[:,2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

unnormalised_acc = (actual*sungaya_dataset.std_3d)+sungaya_dataset.means_3d
ax, line1 = plot_stick_bug(ax, unnormalised_acc, visibility_mask)

unnormalised_est = copy.deepcopy(unnormalised_acc)
unnormalised_est[:,2] = (prediction*std)+mean

# calculate distance from 1 kp
diff  = abs( unnormalised_est[1,2] - unnormalised_acc[1,2] )
for x in range(len(unnormalised_est)):
    unnormalised_est[x,2] = unnormalised_est[x,2] - diff


ax, line2 = plot_stick_bug(ax, unnormalised_est, visibility_mask, True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.title(image[sample])
plt.legend(handles=[line1,line2], loc='upper right')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)
# opens external plot
plt.show()